In [1]:
!pip install transformers

In [2]:
from transformers import pipeline

# pipe = pipeline("[빈칸]", model="sangrimlee/bert-base-multilingual-cased-nsmc")
pipe = pipeline("sentiment-analysis", model="sangrimlee/bert-base-multilingual-cased-nsmc")

def dl_model(input_text):
    output = pipe(input_text)
    if output[0]['label'] == 'positive':
        return 1
    else:
        return 0

# output[0]['label'] == 'positive':
# 예측 결과의 첫 번째 레이블이 'positive'인지 확인

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/932 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/712M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [3]:
import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/tykimos/tykimos.github.io/master/warehouse/dataset/tarr_train.txt",
    filename="tarr_train.txt",
)

('tarr_train.txt', <http.client.HTTPMessage at 0x7c75d007eb30>)

In [4]:
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix

def classify_text(input_text):
    output = dl_model(input_text)
    return output

# 파일을 DataFrame으로 로드
df = pd.read_csv('tarr_train.txt', delimiter='\t')

actual_labels = []
predicted_labels = []

for index, row in df.iterrows():
    comment = row['comment']
    actual_label = row['label']
    predicted_label = classify_text(comment)

    actual_labels.append(actual_label)
    predicted_labels.append(predicted_label)

    if actual_label != predicted_label:
        print("comment : ", comment)
        print("actual class : ", actual_label)
        print("predicted class : ", predicted_label)
        print("---------------")

# 정확도 계산
accuracy = accuracy_score(actual_labels, predicted_labels)
print(f"Accuracy: {accuracy*100:.2f}%")

print(set(actual_labels))
print(set(predicted_labels))

# Confusion matrix 계산
cm = confusion_matrix(actual_labels, predicted_labels, labels=[1, 0])
# Confusion matrix 표현
print("\nConfusion Matrix:")
print("         Predicted:")
print("         긍정    부정")
print("Actual")
print("긍정      {:<5}  {:<5}".format(cm[0][0], cm[0][1]))
print("부정      {:<5}  {:<5}".format(cm[1][0], cm[1][1]))

comment :  안 와봤으면 큰일날뻔; 여기 김치찌개는 진리네 ㅎㅎ
actual class :  1
predicted class :  0
---------------
comment :  사진보다 음식 portion이 좀 작은 거 아닌가요? 😐
actual class :  0
predicted class :  1
---------------
comment :  식사 중에 바퀴벌레가 나와서 기분이 엄청 나빴습니다.
actual class :  0
predicted class :  1
---------------
comment :  ㄴㅇㄱ... 여기서 이런 일을 겪을 줄이야...
actual class :  0
predicted class :  1
---------------
comment :  직원들 태도가 좀 불친절해요. 다음엔 다른 곳을 찾아볼게요.
actual class :  0
predicted class :  1
---------------
comment :  음... 나쁘진 않았지만, 특별히 좋지도 않았어요. 중간이라고 해야하나...
actual class :  1
predicted class :  0
---------------
comment :  그냥저냥? 뭐 크게 나쁘진 않았지만... 😐
actual class :  0
predicted class :  1
---------------
comment :  음식이 너무 짜지 않게 조절해 주셨으면 좋겠어요.
actual class :  0
predicted class :  1
---------------
comment :  저번에 갔을 때보다 맛이 좀 떨어진 것 같아요.
actual class :  0
predicted class :  1
---------------
comment :  떡볶이는 달고, 튀김은 기름져서 다시 오진 않을 것 같아요.
actual class :  0
predicted class :  1
---------------
comment :  여기 직원분들이 좀 불친절하시던데요?

In [5]:
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/tykimos/tykimos.github.io/master/warehouse/dataset/tarr_sample_submit.txt",
    filename="tarr_sample_submit.txt",
)

('tarr_sample_submit.txt', <http.client.HTTPMessage at 0x7c75d007ee00>)

In [6]:
# 파일을 DataFrame으로 로드
df_submit = pd.read_csv('tarr_sample_submit.txt', delimiter='\t')

predicted_labels = []

# 각 row를 순회하며 코멘트를 분류
for index, row in df_submit.iterrows():
    comment = row['comment']
    predicted_label = classify_text(comment)
    predicted_labels.append(predicted_label)

# 예측된 레이블을 DataFrame에 추가
df_submit['label'] = predicted_labels

# 결과를 tarr_my_submit.txt로 저장
df_submit[['id', 'comment', 'label']].to_csv('tarr_my_submit.txt', sep='\t', index=False)

In [7]:
df_submit

,id,comment,label
0,1,완전 내 스타일이에요! 가격도 적당하고 위치도 좋고👌,1
1,2,맛있긴 한데 양이 너무 적어서 좀... ㅠ,0
2,3,완전 내 스타일이에요 ㅠㅠ 여기 매장 분위기도 이쁨,1
3,4,한국의 전통 음식을 잘 표현한 것 같아요. 향토음식의 정취가 느껴져 좋았습니다.,1
4,5,서빙하는 분이 좀 불친절해서 기분이 좀 그랬어요.,0
...,...,...,...
95,96,가성비 최고! 이 가격에 이런 맛은 정말 만족스러워요.,1
96,97,와... 여기김치... 말도안됨... ㅁㅊ...,1
97,98,주문한 지 40분 넘게 기다려서 음식 나왔네요...,0
98,99,"아이들이랑 왔는데, 키즈 메뉴도 생각보다 맛있었어요!",1
